In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "report_template_header",
        "colab_type": "text"
      },
      "source": [
        "# Market Research Report Template Development\n",
        "## Indian Stock Market Research System v1.0\n",
        "### Date: 2022\n",
        "\n",
        "This notebook is used for developing and testing report templates for the Indian market research system."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "setup_section",
        "colab_type": "text"
      },
      "source": [
        "## Setup and Dependencies"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "install_dependencies",
        "colab_type": "code"
      },
      "outputs": [],
      "source": [
        "# Install required packages\n",
        "!pip install yfinance pandas numpy matplotlib seaborn plotly reportlab fpdf2\n",
        "!pip install ta-lib-binary jinja2 datetime pytz"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "import_libraries",
        "colab_type": "code"
      },
      "outputs": [],
      "source": [
        "import yfinance as yf\n",
        "import pandas as pd\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "from datetime import datetime, timedelta\n",
        "import pytz\n",
        "from jinja2 import Template\n",
        "from fpdf import FPDF\n",
        "import io\n",
        "import base64\n",
        "import warnings\n",
        "warnings.filterwarnings('ignore')\n",
        "\n",
        "# Set style for plots\n",
        "plt.style.use('seaborn-v0_8')\n",
        "sns.set_palette(\"husl\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "indian_stocks_config",
        "colab_type": "text"
      },
      "source": [
        "## Indian Stock Market Configuration"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "stock_symbols",
        "colab_type": "code"
      },
      "outputs": [],
      "source": [
        "# Top Indian stocks for analysis (NSE symbols)\n",
        "INDIAN_STOCKS = {\n",
        "    'Large Cap': {\n",
        "        'RELIANCE.NS': 'Reliance Industries',\n",
        "        'TCS.NS': 'Tata Consultancy Services',\n",
        "        'HDFCBANK.NS': 'HDFC Bank',\n",
        "        'INFY.NS': 'Infosys',\n",
        "        'HINDUNILVR.NS': 'Hindustan Unilever',\n",
        "        'ICICIBANK.NS': 'ICICI Bank',\n",
        "        'SBIN.NS': 'State Bank of India',\n",
        "        'BHARTIARTL.NS': 'Bharti Airtel',\n",
        "        'ITC.NS': 'ITC Limited',\n",
        "        'KOTAKBANK.NS': 'Kotak Mahindra Bank'\n",
        "    },\n",
        "    'Mid Cap': {\n",
        "        'ADANIPORTS.NS': 'Adani Ports',\n",
        "        'HCLTECH.NS': 'HCL Technologies',\n",
        "        'WIPRO.NS': 'Wipro',\n",
        "        'MARUTI.NS': 'Maruti Suzuki',\n",
        "        'TITAN.NS': 'Titan Company'\n",
        "    }\n",
        "}\n",
        "\n",
        "# Indian Market Indices\n",
        "INDIAN_INDICES = {\n",
        "    '^NSEI': 'Nifty 50',\n",
        "    '^BSESN': 'BSE Sensex',\n",
        "    '^NSEBANK': 'Nifty Bank'\n",
        "}\n",
        "\n",
        "# Combine all symbols\n",
        "ALL_STOCKS = {}\n",
        "for category in INDIAN_STOCKS.values():\n",
        "    ALL_STOCKS.update(category)\n",
        "\n",
        "print(f\"Configured {len(ALL_STOCKS)} Indian stocks for analysis\")\n",
        "print(f\"Configured {len(INDIAN_INDICES)} Indian indices for analysis\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "template_functions",
        "colab_type": "text"
      },
      "source": [
        "## Report Template Functions"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "data_fetcher",
        "colab_type": "code"
      },
      "outputs": [],
      "source": [
        "def fetch_indian_market_data(symbols, period='1mo'):\n",
        "    \"\"\"\n",
        "    Fetch market data for Indian stocks and indices\n",
        "    \"\"\"\n",
        "    data = {}\n",
        "    for symbol in symbols:\n",
        "        try:\n",
        "            ticker = yf.Ticker(symbol)\n",
        "            hist = ticker.history(period=period)\n",
        "            info = ticker.info\n",
        "            \n",
        "            data[symbol] = {\n",
        "                'history': hist,\n",
        "                'info': info,\n",
        "                'current_price': hist['Close'].iloc[-1] if not hist.empty else 0,\n",
        "                'change_1d': ((hist['Close'].iloc[-1] / hist['Close'].iloc[-2] - 1) * 100) if len(hist) > 1 else 0\n",
        "            }\n",
        "        except Exception as e:\n",
        "            print(f\"Error fetching {symbol}: {e}\")\n",
        "            data[symbol] = None\n",
        "    \n",
        "    return data\n",
        "\n",
        "def calculate_technical_indicators(df):\n",
        "    \"\"\"\n",
        "    Calculate basic technical indicators\n",
        "    \"\"\"\n",
        "    if df.empty:\n",
        "        return df\n",
        "    \n",
        "    # Simple Moving Averages\n",
        "    df['SMA_20'] = df['Close'].rolling(window=20).mean()\n",
        "    df['SMA_50'] = df['Close'].rolling(window=50).mean()\n",
        "    \n",
        "    # RSI calculation\n",
        "    delta = df['Close'].diff()\n",
        "    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()\n",
        "    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()\n",
        "    rs = gain / loss\n",
        "    df['RSI'] = 100 - (100 / (1 + rs))\n",
        "    \n",
        "    # Bollinger Bands\n",
        "    bb_period = 20\n",
        "    bb_std = 2\n",
        "    df['BB_Middle'] = df['Close'].rolling(window=bb_period).mean()\n",
        "    bb_std_dev = df['Close'].rolling(window=bb_period).std()\n",
        "    df['BB_Upper'] = df['BB_Middle'] + (bb_std_dev * bb_std)\n",
        "    df['BB_Lower'] = df['BB_Middle'] - (bb_std_dev * bb_std)\n",
        "    \n",
        "    return df"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "chart_generators",
        "colab_type": "code"
      },
      "outputs": [],
      "source": [
        "def create_price_chart(df, title, symbol):\n",
        "    \"\"\"\n",
        "    Create price chart with technical indicators\n",
        "    \"\"\"\n",
        "    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), height_ratios=[3, 1])\n",
        "    \n",
        "    # Price and moving averages\n",
        "    ax1.plot(df.index, df['Close'], label='Close Price', linewidth=2)\n",
        "    ax1.plot(df.index, df['SMA_20'], label='SMA 20', alpha=0.7)\n",
        "    ax1.plot(df.index, df['SMA_50'], label='SMA 50', alpha=0.7)\n",
        "    \n",
        "    # Bollinger Bands\n",
        "    ax1.fill_between(df.index, df['BB_Upper'], df['BB_Lower'], alpha=0.2, label='Bollinger Bands')\n",
        "    \n",
        "    ax1.set_title(f'{title} - Price Chart with Technical Indicators', fontsize=14, fontweight='bold')\n",
        "    ax1.set_ylabel('Price (₹)')\n",
        "    ax1.legend()\n",
        "    ax1.grid(True, alpha=0.3)\n",
        "    \n",
        "    # RSI subplot\n",
        "    ax2.plot(df.index, df['RSI'], color='purple', linewidth=2)\n",
        "    ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')\n",
        "    ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')\n",
        "    ax2.set_title('RSI (Relative Strength Index)')\n",
        "    ax2.set_ylabel('RSI')\n",
        "    ax2.set_xlabel('Date')\n",
        "    ax2.legend()\n",
        "    ax2.grid(True, alpha=0.3)\n",
        "    ax2.set_ylim(0, 100)\n",
        "    \n",
        "    plt.tight_layout()\n",
        "    return fig\n",
        "\n",
        "def create_market_overview_chart(data):\n",
        "    \"\"\"\n",
        "    Create market overview chart\n",
        "    \"\"\"\n",
        "    symbols = []\n",
        "    changes = []\n",
        "    names = []\n",
        "    \n",
        "    for symbol, stock_data in data.items():\n",
        "        if stock_data and not pd.isna(stock_data['change_1d']):\n",
        "            symbols.append(symbol.replace('.NS', ''))\n",
        "            changes.append(stock_data['change_1d'])\n",
        "            names.append(ALL_STOCKS.get(symbol, symbol))\n",
        "    \n",
        "    fig, ax = plt.subplots(figsize=(12, 8))\n",
        "    colors = ['green' if x > 0 else 'red' for x in changes]\n",
        "    \n",
        "    bars = ax.barh(symbols, changes, color=colors, alpha=0.7)\n",
        "    ax.set_xlabel('1-Day Change (%)')\n",
        "    ax.set_title('Indian Stock Market - Daily Performance Overview', fontsize=14, fontweight='bold')\n",
        "    ax.axvline(x=0, color='black', linewidth=0.8)\n",
        "    ax.grid(axis='x', alpha=0.3)\n",
        "    \n",
        "    # Add value labels on bars\n",
        "    for bar, change in zip(bars, changes):\n",
        "        width = bar.get_width()\n",
        "        ax.text(width + (0.1 if width > 0 else -0.1), bar.get_y() + bar.get_height()/2,\n",
        "                f'{change:.1f}%', ha='left' if width > 0 else 'right', va='center')\n",
        "    \n",
        "    plt.tight_layout()\n",
        "    return fig"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "html_template",
        "colab_type": "text"
      },
      "source": [
        "## HTML Report Template"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "html_template_code",
        "colab_type": "code"
      },
      "outputs": [],
      "source": [
        "HTML_TEMPLATE = \"\"\"\n",
        "<!DOCTYPE html>\n",
        "<html>\n",
        "<head>\n",
        "    <title>Indian Market Research Report - {{ report_date }}</title>\n",
        "    <style>\n",
        "        body {\n",
        "            font-family: 'Arial', sans-serif;\n",
        "            margin: 40px;\n",
        "            background-color: #f8f9fa;\n",
        "        }\n",
        "        .header {\n",
        "            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);\n",
        "            color: white;\n",
        "            padding: 30px;\n",
        "            border-radius: 10px;\n",
        "            text-align: center;\n",
        "            margin-bottom: 30px;\n",
        "        }\n",
        "        .section {\n",
        "            background: white;\n",
        "            margin: 20px 0;\n",
        "            padding: 25px;\n",
        "            border-radius: 8px;\n",
        "            box-shadow: 0 2px 10px rgba(0,0,0,0.1);\n",
        "        }\n",
        "        .metric-card {\n",
        "            display: inline-block;\n",
        "            background: #f8f9fa;\n",
        "            padding: 15px;\n",
        "            margin: 10px;\n",
        "            border-radius: 8px;\n",
        "            text-align: center;\n",
        "            min-width: 150px;\n",
        "        }\n",
        "        .positive { color: #28a745; }\n",
        "        .negative { color: #dc3545; }\n",
        "        .chart-container {\n",
        "            text-align: center;\n",
        "            margin: 20px 0;\n",
        "        }\n",
        "        table {\n",
        "            width: 100%;\n",
        "            border-collapse: collapse;\n",
        "            margin: 15px 0;\n",
        "        }\n",
        "        th, td {\n",
        "            padding: 12px;\n",
        "            text-align: left;\n",
        "            border-bottom: 1px solid #ddd;\n",
        "        }\n",
        "        th {\n",
        "            background-color: #f8f9fa;\n",
        "            font-weight: bold;\n",
        "        }\n",
        "    </style>\n",
        "</head>\n",
        "<body>\n",
        "    <div class=\"header\">\n",
        "        <h1>Indian Market Research Report</h1>\n",
        "        <h2>{{ report_date }}</h2>\n",
        "        <p>Independent Market Research System v1.0</p>\n",
        "    </div>\n",
        "    \n",
        "    <div class=\"section\">\n",
        "        <h2>Executive Summary</h2>\n",
        "        <p>{{ executive_summary }}</p>\n",
        "        \n",
        "        <div class=\"metric-card\">\n",
        "            <h3>Market Sentiment</h3>\n",
        "            <p class=\"{{ sentiment_class }}\">{{ market_sentiment }}</p>\n",
        "        </div>\n",
        "        \n",
        "        <div class=\"metric-card\">\n",
        "            <h3>Volatility</h3>\n",
        "            <p>{{ volatility_level }}</p>\n",
        "        </div>\n",
        "        \n",
        "        <div class=\"metric-card\">\n",
        "            <h3>Trading Volume</h3>\n",
        "            <p>{{ volume_trend }}</p>\n",
        "        </div>\n",
        "    </div>\n",
        "    \n",
        "    <div class=\"section\">\n",
        "        <h2>Market Overview</h2>\n",
        "        <div class=\"chart-container\">\n",
        "            <img src=\"data:image/png;base64,{{ market_chart }}\" style=\"max-width: 100%; height: auto;\">\n",
        "        </div>\n",
        "    </div>\n",
        "    \n",
        "    <div class=\"section\">\n",
        "        <h2>Top Performers</h2>\n",
        "        <table>\n",
        "            <tr>\n",
        "                <th>Stock</th>\n",
        "                <th>Current Price (₹)</th>\n",
        "                <th>1-Day Change</th>\n",
        "                <th>Status</th>\n",
        "            </tr>\n",
        "            {% for stock in top_stocks %}\n",
        "            <tr>\n",
        "                <td>{{ stock.name }}</td>\n",
        "                <td>{{ stock.price }}</td>\n",
        "                <td class=\"{% if stock.change > 0 %}positive{% else %}negative{% endif %}\">\n",
        "                    {{ stock.change }}%\n",
        "                </td>\n",
        "                <td>{{ stock.status }}</td>\n",
        "            </tr>\n",
        "            {% endfor %}\n",
        "        </table>\n",
        "    </div>\n",
        "    \n",
        "    <div class=\"section\">\n",
        "        <h2>Technical Analysis Highlights</h2>\n",
        "        <ul>\n",
        "            {% for highlight in technical_highlights %}\n",
        "            <li>{{ highlight }}</li>\n",
        "            {% endfor %}\n",
        "        </ul>\n",
        "    </div>\n",
        "    \n",
        "    <div class=\"section\">\n",
        "        <h2>Market Outlook</h2>\n",
        "        <p>{{ market_outlook }}</p>\n",
        "    </div>\n",
        "    \n",
        "    <div class=\"section\" style=\"font-size: 0.9em; color: #666;\">\n",
        "        <p><strong>Disclaimer:</strong> This report is for educational and research purposes only. Not investment advice.</p>\n",
        "        <p><strong>Generated:</strong> {{ generation_time }} | <strong>System:</strong> Market Research v1.0</p>\n",
        "    </div>\n",
        "</body>\n",
        "</html>\n",
        "\"\"\""
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "report_generator",
        "colab_type": "text"
      },
      "source": [
        "## Report Generation Functions"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "report_functions",
        "colab_type": "code"
      },
      "outputs": [],
      "source": [
        "def generate_market_report():\n",
        "    \"\"\"\n",
        "    Generate comprehensive market report\n",
        "    \"\"\"\n",
        "    print(\"Fetching Indian market data...\")\n",
        "    \n",
        "    # Fetch data for top stocks\n",
        "    stock_symbols = list(ALL_STOCKS.keys())[:10]  # Top 10 for demo\n",
        "    market_data = fetch_indian_market_data(stock_symbols, period='3mo')\n",
        "    \n",
        "    # Process data\n",
        "    processed_stocks = []\n",
        "    total_change = 0\n",
        "    valid_stocks = 0\n",
        "    \n",
        "    for symbol, data in market_data.items():\n",
        "        if data and data['current_price'] > 0:\n",
        "            processed_stocks.append({\n",
        "                'name': ALL_STOCKS.get(symbol, symbol),\n",
        "                'symbol': symbol,\n",
        "                'price': f\"{data['current_price']:.2f}\",\n",
        "                'change': round(data['change_1d'], 2),\n",
        "                'status': 'Bullish' if data['change_1d'] > 2 else 'Bearish' if data['change_1d'] < -2 else 'Neutral'\n",
        "            })\n",
        "            total_change += data['change_1d']\n",
        "            valid_stocks += 1\n",
        "    \n",
        "    # Sort by performance\n",
        "    processed_stocks.sort(key=lambda x: x['change'], reverse=True)\n",
        "    \n",
        "    # Calculate market sentiment\n",
        "    avg_change = total_change / valid_stocks if valid_stocks > 0 else 0\n",
        "    sentiment = 'Positive' if avg_change > 0.5 else 'Negative' if avg_change < -0.5 else 'Neutral'\n",
        "    sentiment_class = 'positive' if avg_change > 0 else 'negative'\n",
        "    \n",
        "    # Generate market overview chart\n",
        "    fig = create_market_overview_chart(market_data)\n",
        "    \n",
        "    # Convert plot to base64\n",
        "    img_buffer = io.BytesIO()\n",
        "    fig.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')\n",
        "    img_buffer.seek(0)\n",
        "    chart_base64 = base64.b64encode(img_buffer.getvalue()).decode()\n",
        "    plt.close(fig)\n",
        "    \n",
        "    # Generate technical highlights\n",
        "    highlights = [\n",
        "        f\"Average market change: {avg_change:.2f}%\",\n",
        "        f\"Best performer: {processed_stocks[0]['name']} (+{processed_stocks[0]['change']}%)\",\n",
        "        f\"Worst performer: {processed_stocks[-1]['name']} ({processed_stocks[-1]['change']}%)\",\n",
        "        f\"Market breadth: {sum(1 for s in processed_stocks if s['change'] > 0)}/{len(processed_stocks)} stocks positive\"\n",
        "    ]\n",
        "    \n",
        "    # Prepare template data\n",
        "    template_data = {\n",
        "        'report_date': datetime.now(pytz.timezone('Asia/Kolkata')).strftime('%B %d, %Y'),\n",
        "        'executive_summary': f\"Indian equity markets showed {sentiment.lower()} sentiment with an average change of {avg_change:.2f}% across major stocks.\",\n",
        "        'market_sentiment': sentiment,\n",
        "        'sentiment_class': sentiment_class,\n",
        "        'volatility_level': 'Moderate',\n",
        "        'volume_trend': 'Normal',\n",
        "        'market_chart': chart_base64,\n",
        "        'top_stocks': processed_stocks[:8],  # Top 8 for display\n",
        "        'technical_highlights': highlights,\n",
        "        'market_outlook': f\"Based on current technical indicators, the market shows {sentiment.lower()} bias. Key levels to watch for Nifty 50 and major stocks.\",\n",
        "        'generation_time': datetime.now(pytz.timezone('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S IST')\n",
        "    }\n",
        "    \n",
        "    return template_data\n",
        "\n",
        "def create_html_report(template_data):\n",
        "    \"\"\"\n",
        "    Create HTML report from template\n",
        "    \"\"\"\n",
        "    template = Template(HTML_TEMPLATE)\n",
        "    html_content = template.render(**template_data)\n",
        "    return html_content"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "test_report",
        "colab_type": "text"
      },
      "source": [
        "## Test Report Generation"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "generate_test_report",
        "colab_type": "code"
      },
      "outputs": [],
      "source": [
        "# Generate test report\n",
        "print(\"Generating Indian Market Research Report...\")\n",
        "template_data = generate_market_report()\n",
        "\n",
        "# Create HTML report\n",
        "html_report = create_html_report(template_data)\n",
        "\n",
        "# Save report\n",
        "report_filename = f\"indian_market_report_{datetime.now().strftime('%Y%m%d')}.html\"\n",
        "with open(report_filename, 'w', encoding='utf-8') as f:\n",
        "    f.write(html_report)\n",
        "\n",
        "print(f\"Report generated successfully: {report_filename}\")\n",
        "print(f\"Report contains data for {len(template_data['top_stocks'])} stocks\")\n",
        "print(f\"Market sentiment: {template_data['market_sentiment']}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "template_customization",
        "colab_type": "text"
      },
      "source": [
        "## Template Customization Options"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "customization_functions",
        "colab_type": "code"
      },
      "outputs": [],
      "source": [
        "def create_sector_specific_template():\n",
        "    \"\"\"\n",
        "    Create sector-specific report template\n",
        "    \"\"\"\n",
        "    sector_template = \"\"\"\n",
        "    <div class=\"section\">\n",
        "        <h2>Sector Analysis - {{ sector_name }}</h2>\n",
        "        <div class=\"metric-card\">\n",
        "            <h4>Sector Performance</h4>\n",
        "            <p class=\"{{ sector_class }}\">{{ sector_change }}%</p>\n",
        "        </div>\n",
        "        <table>\n",
        "            <tr><th>Stock</th><th>Price</th><th>Change</th><th>Volume</th></tr>\n",
        "            {% for stock in sector_stocks %}\n",
        "            <tr>\n",
        "                <td>{{ stock.name }}</td>\n",
        "                <td>₹{{ stock.price }}</td>\n",
        "                <td class=\"{% if stock.change > 0 %}positive{% else %}negative{% endif %}\">\n",
        "                    {{ stock.change }}%\n",
        "                </td>\n",
        "                <td>{{ stock.volume }}</td>\n",
        "            </tr>\n",
        "            {% endfor %}\n",
        "        </table>\n",
        "    </div>\n",
        "    \"\"\"\n",
        "    return sector_template\n",
        "\n",
        "def create_weekly_template():\n",
        "    \"\"\"\n",
        "    Create weekly report template\n",
        "    \"\"\"\n",
        "    weekly_template = \"\"\"\n",
        "    <div class=\"section\">\n",
        "        <h2>Weekly Market Summary</h2>\n",
        "        <p><strong>Week of:</strong> {{ week_start }} to {{ week_end }}</p>\n",
        "        \n",
        "        <div class=\"metric-card\">\n",
        "            <h4>Weekly Change</h4>\n",
        "            <p class=\"{{ weekly_class }}\">{{ weekly_change }}%</p>\n",
        "        </div>\n",
        "        \n",
        "        <h3>Key Events This Week</h3>\n",
        "